In [2]:
import psycopg2
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from glob import glob

In [3]:
# db helper functions
def get_table_data(table_name):
    conn = psycopg2.connect(database="ProdDB",
                            host="localhost",
                            user="postgres",
                            password="manu1234",
                            port="5432")
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    df = pd.DataFrame(cursor, columns=[desc[0] for desc in cursor.description])
    conn.close()
    return df

def get_values_data(companyid):
    conn = psycopg2.connect(database="ProdDB",
                            host="localhost",
                            user="postgres",
                            password="manu1234",
                            port="5432")
    cursor = conn.cursor()
    query = f"WITH FilteredOrders AS (SELECT * FROM values WHERE values.companyid = {companyid}) SELECT * FROM FilteredOrders AS fo RIGHT JOIN variables ON fo.variableid = variables.id ORDER by variables.id"
    cursor.execute(query)
    df = pd.DataFrame(cursor, columns=[desc[0] for desc in cursor.description])
    conn.close()
    return df

In [4]:
companies_df = get_table_data('companies')
msgs = []
full_tagged_tables = []
for _, company_row in tqdm(companies_df.iterrows(), total=companies_df.shape[0]):
    values_df = get_values_data(company_row['id'])
    if values_df.empty:
        continue
    periods_set = set(values_df['period'])
    for table_name, table_df in values_df.groupby('table'):
        for varid, grp in table_df.groupby('variableid'):
            missing_values = periods_set - set(grp['period'])
            if missing_values:
                break
        else:
            full_tagged_tables.append(table_df)
            continue
        msgs.append(f"company: {company_row['Name']}, table:{table_name} has empty values")

100%|██████████| 503/503 [00:16<00:00, 30.77it/s]


In [5]:
len(full_tagged_tables)

1088

In [6]:
index = 34
company = companies_df[companies_df['id']==full_tagged_tables[index].iloc[0]['companyid']]
print(company['Name'].iloc[0])
print(company['Symbol'].iloc[0])
full_tagged_tables[index]

CenterPoint Energy
CNP


,id,value,scale,period,variableid,companyid,documenttype,type,id,variable,table,xbrlids,type,formula
122,8803,1.638000e+09,1,2019-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
123,27699,2.136000e+09,1,2018-12-31,17,110,10K,extracted,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
124,17522,2.136000e+09,1,2018-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
125,17521,1.421000e+09,1,2017-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
126,27711,1.810000e+09,1,2022-12-31,17,110,10K,extracted,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
127,8833,1.421000e+09,1,2017-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
128,19929,1.421000e+09,1,2017-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
129,19932,2.136000e+09,1,2018-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
130,19935,1.638000e+09,1,2019-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
131,19938,1.995000e+09,1,2020-12-31,17,110,10K,None,17,Operating Cash Flow,cash flow,[us-gaap:NetCashProvidedByUsedInOperatingActiv...,display,None
